In [1]:
import numpy as np
from numpy.random import permutation

import os
import glob
import cv2
import math
import sys

import pandas as pd

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD
from keras.utils import np_utils

import tensorflow as tf


import warnings
warnings.filterwarnings('ignore')

In [2]:
# GPU 강제 할당
# 사용 가능한 GPU 목록을 가져온다.
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용할 수 있도록 설정한다.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# 랜덤시드
np.random.seed(1)

In [4]:
# 사용하는 이미지의 사이즈
img_rows = 224
img_cols = 224

In [5]:
# 실행 모드
run_type = 'train'
# run_type = 'test'

### 학습

In [6]:
# 9층 신경만 생성함수
def layer_9_model():
    # 모델 생성
    model = Sequential()
    
    # Convolution & MaxPooling 층
    # 2개의 층을 하나의 묶음으로 봄
    # 1층
    model.add(Conv2D(32, kernel_size=(3,3), padding='same',
                    activation='linear', input_shape=(img_rows, img_cols,3)))
    model.add(LeakyReLU(alpha=0.3))
    # 2층
    model.add(Conv2D(32, kernel_size=(3,3), padding='same',
                   activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    # 2개의 층을 하나의 묶음으로 봄
    # 3층
    model.add(Conv2D(64,kernel_size=(3,3), padding='same',
                    activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    # 4층
    model.add(Conv2D(64, kernel_size=(3,3),padding='same',
                    activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    
    # 5층
    model.add(Conv2D(128, kernel_size=(3,3), padding='same',
                    activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    
    # 6층
    model.add(Conv2D(128, kernel_size=(3,3), padding='same',
                    activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    
    # Flatten 층 (2차원 -> 1차원)
    model.add(Flatten())
    
    # 전결합 & DropOut 층
    model.add(Dense(1024, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5))
    
    model.add(Dense(1024, activation='linear'))
    model.add(LeakyReLU(alpha=0.3))
    model.add(Dropout(0.5))
    

    # 출력층 (결과가 6종류이므로..)
    model.add(Dense(6, activation='softmax'))
    
    # 컴파일 
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                 metrics=['accuracy'])
    
    # 구성확인
    model.summary()
    
    return model

In [7]:
# 이미지 1장을 읽어오고 리사이징한다.
def get_img(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_rows, img_cols))
    return resized

In [8]:
# 학습 데이터를 읽어오는 함수
def read_train_data(ho=0, kind='train'):
    # 학습용 입력데이터를 담을 리스트
    train_data = []
    # 학습용 결과데이터를 담을 리스트
    train_target = []
    
    # 결과 종류의 수만큼 반복한다.(비행기, 오토바이, 등등)
    for j in range(0,6):
        # 이미지의 경로
        path = 'data/Caltech-101/'
        path += f'{kind}/{ho}/*/{j}/*.jpg' # ex) train/0/0/*.jpg
        
        # 파일 목록을 가져온다.
        files = sorted(glob.glob(path))
        # print(files)
        
        # 파일의 수만큼 반복한다.
        for f1 in files:
            # 파일 이름을 가져온다.
            file_base = os.path.basename(f1)
            
            # 이미지 1장을 읽어온다.
            img = get_img(f1)
            img = np.array(img, dtype=np.float32)
            
            # 데이터 정규화
            img -= np.mean(img)
            img /= np.std(img)
            
            # 리스트에 담는다.
            train_data.append(img)
            train_target.append(j)
    
    # 읽어들인 데이터를 numpy의 array로 변환
    train_data = np.array(train_data, dtype=np.float32)
    train_target = np.array(train_target, dtype=np.uint8)
    
    # target을 원핫 인코딩한다.
    # 예) 1 -> 0,1,0,0,0,0
    train_target = np_utils.to_categorical(train_target, 6)
    
    # 데이터를 섞는다.
    perm = permutation(len(train_target))
    train_data = train_data[perm]
    train_target = train_target[perm]
    
    return train_data, train_target
    

In [9]:
# 모델의 구조와 가중치를 저장한다.
def save_model(model, ho, modelStr=''):
    
    # 모델 객체를 json 형식으로 변환한다.
    json_string = model.to_json()
    
    # cache 폴더가 없으면 만들어준다.
    if not os.path.exists('cache'):
        os.makedirs('cache')
        
    # 모델 구조를 저장하기 위한 파일명
    json_name = f'architecture_{modelStr}_{ho}.json'
    
    # 모델 구조를 저장한다.
    with open(os.path.join('cache', json_name),'w') as fp:
        fp.write(json_string)

In [10]:
# 학습 함수
def run_train(modelStr=''):
    # HoldOut을 두번 수행한다.
    for ho in range(2):
        # 모델을 생성한다.
        model = layer_9_model()
        
        # 학습 데이터를 읽어온다. 함수호출
        t_data, t_target = read_train_data(ho,'train') 
        # 검증 데이터를 읽어온다.
        v_data, v_target = read_train_data(ho,'valid')
        
        # 매 epoch마다 모델을 저장하기 위한 callback을 생성한다.
        cp = ModelCheckpoint(f'cache/model_weights_{modelStr}_{ho}_' + '{epoch:02d}.h5',
                            monitor='val_loss', save_best_only=False)
        
        # train 실행
        model.fit(t_data, t_target, batch_size=16, epochs=40,
                 validation_data=(v_data, v_target), shuffle=True, callbacks=[cp])
        
        # 모델 구조 저장
        save_model(model, ho, modelStr)

### 예측

### 실행

In [11]:
# CPU 실행
# with tf.device('/CPU:0'):
#     if run_type == 'train':
#     run_train('9_Layer_CNN')
# elif run_type == 'test':
#     pass

# GPU 실행
# GPU 메모리 부족 에러 Failed to get convolution algorithm

if run_type == 'train':
    run_train('9_Layer_CNN')
elif run_type == 'test':
    pass

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 112, 112, 64)      0

82/82 [==============================] - 9s 112ms/step - loss: 0.4181 - accuracy: 0.9496 - val_loss: 8.4377 - val_accuracy: 0.6889
Epoch 35/40
82/82 [==============================] - 9s 111ms/step - loss: 0.5385 - accuracy: 0.9535 - val_loss: 10.0031 - val_accuracy: 0.6897
Epoch 36/40
82/82 [==============================] - 9s 111ms/step - loss: 0.6860 - accuracy: 0.9387 - val_loss: 9.6934 - val_accuracy: 0.6966
Epoch 37/40
82/82 [==============================] - 9s 112ms/step - loss: 0.2352 - accuracy: 0.9727 - val_loss: 10.9085 - val_accuracy: 0.6897
Epoch 38/40
82/82 [==============================] - 9s 111ms/step - loss: 0.6086 - accuracy: 0.9469 - val_loss: 10.6230 - val_accuracy: 0.6743
Epoch 39/40
82/82 [==============================] - 9s 111ms/step - loss: 0.2984 - accuracy: 0.9699 - val_loss: 10.3207 - val_accuracy: 0.6866
Epoch 40/40
82/82 [==============================] - 9s 111ms/step - loss: 0.4207 - accuracy: 0.9662 - val_loss: 9.6892 - val_accuracy: 0.6889
Model: 

82/82 [==============================] - 9s 112ms/step - loss: 0.7163 - accuracy: 0.7813 - val_loss: 3.0640 - val_accuracy: 0.5318
Epoch 28/40
82/82 [==============================] - 9s 112ms/step - loss: 1.1130 - accuracy: 0.7306 - val_loss: 1.4249 - val_accuracy: 0.7372
Epoch 29/40
82/82 [==============================] - 9s 111ms/step - loss: 0.6454 - accuracy: 0.8270 - val_loss: 1.8013 - val_accuracy: 0.7625
Epoch 30/40
82/82 [==============================] - 9s 112ms/step - loss: 0.6193 - accuracy: 0.8248 - val_loss: 1.4759 - val_accuracy: 0.7525
Epoch 31/40
82/82 [==============================] - 9s 112ms/step - loss: 0.4438 - accuracy: 0.8686 - val_loss: 1.7775 - val_accuracy: 0.7479
Epoch 32/40
82/82 [==============================] - 9s 111ms/step - loss: 0.3504 - accuracy: 0.8813 - val_loss: 1.8401 - val_accuracy: 0.7142
Epoch 33/40
82/82 [==============================] - 9s 112ms/step - loss: 0.4515 - accuracy: 0.8548 - val_loss: 1.7056 - val_accuracy: 0.7441
Epoch 34/40